In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [1, 13]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [2]:
model.features[27]

QuantConv2d(
  8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)

In [ ]:
# This cell won't be given, but students will complete the training

def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [7, 26, 39]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1  

lr = 1e-2
weight_decay = 1e-3
epochs = 11
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name) + '_modified'
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [3]:
PATH = "result/VGG16_quant_modified/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)



Test set: Accuracy: 8335/10000 (83%)



In [4]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()

for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print("prehooked")
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)
# print("1st convolution's input size:", save_output.outputs[0][0].size())
# print("2nd convolution's input size:", save_output.outputs[1][0].size())

prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked


In [5]:
i=0
for iter, val in enumerate(model.features):
    if isinstance(val,QuantConv2d):
        print (i, iter, val)
        print ("\n")
        i=i+1
        

0 0 QuantConv2d(
  3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)


1 3 QuantConv2d(
  64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)


2 7 QuantConv2d(
  64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)


3 10 QuantConv2d(
  128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)


4 14 QuantConv2d(
  128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)


5 17 QuantConv2d(
  256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)


6 20 QuantConv2d(
  256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)


7 24 QuantConv2d(
  256, 8, kernel_size=(3, 3), stride=(1, 1), p

In [6]:
for i in range(13):
    print(i , save_output.outputs[i][0][0].size())

0 torch.Size([3, 32, 32])
1 torch.Size([64, 32, 32])
2 torch.Size([64, 16, 16])
3 torch.Size([128, 16, 16])
4 torch.Size([128, 8, 8])
5 torch.Size([256, 8, 8])
6 torch.Size([256, 8, 8])
7 torch.Size([256, 4, 4])
8 torch.Size([8, 4, 4])
9 torch.Size([8, 4, 4])
10 torch.Size([512, 2, 2])
11 torch.Size([512, 2, 2])
12 torch.Size([512, 2, 2])


In [7]:
w_bit = 4
weight_q = model.features[27].weight_q # quantized value is stored during the training
w_alpha = model.features[27].weight_quant.wgt_alpha
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q/w_delta
#print(weight_int) # you should see clean integer numbers

In [8]:
x_bit = 4
x = save_output.outputs[8][0] # input of the 2nd conv layer
x_alpha = model.features[27].act_alpha
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x



x_int = x_q/x_delta
#print(x_int) # you should see clean integer numbers 


In [9]:
x_int.size()

torch.Size([128, 8, 4, 4])

In [10]:
######## Padding before Convolution #######
x_pad = torch.zeros(128, 8, 6, 6).cuda()
# a_pad.size() = [64, 32+2pad, 32+2pad]
x_pad[ : ,  :, 1:5, 1:5] = x_int.cuda()

In [11]:
x_pad.size()

torch.Size([128, 8, 6, 6])

In [12]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)


output_int =  conv_int(x_pad)
output_recovered = output_int*w_delta*x_delta
#print(output_recovered)

In [13]:
output_int.size()

torch.Size([128, 8, 4, 4])

In [14]:
# conv_ref = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, bias = False)
conv_ref = torch.nn.Conv2d(in_channels = 8, out_channels=8, padding=(1, 1), kernel_size = 3, bias = False)
conv_ref.weight = model.features[27].weight_q

output_ref = conv_ref(x)
#print(output_ref)

In [15]:
#  padding=(1, 1),
output_ref.size()

torch.Size([128, 8, 4, 4])

In [16]:
difference = abs( output_ref[0] - output_recovered[0] )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model


tensor(0.3014, device='cuda:0', grad_fn=<MeanBackward0>)


In [17]:
X = x_pad[0] 
X = torch.reshape(X, (X.size(0), -1))

bit_precision = 4
file = open('activation.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(int(X[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(X_bin[k])        
#         file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file

In [18]:
bit_precision = 4
file = open('activation_dec.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_dec = str(int(X[7-j,i].item()+0.001))
        file.write(X_dec)        
        file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file

In [19]:
X[:,7]

tensor([0., 1., 0., 1., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward>)

In [20]:
X = x_pad[0] 
# X = torch.reshape(X, (4,72,-1))
X = torch.reshape(X, (X.size(0), -1))
X.size()

torch.Size([8, 36])

In [21]:
weight_int.size() # 8, 8 , 3, 3
W = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))
W.size() # 8, 8, 9

bit_precision = 4
file = open('weight.txt', 'w') #write to file
file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
file.write('#................#\n')


W_temp=0
for kij in range(9):
    for i in range(W.size(0)):  # Column #
        for j in range(W.size(1)): # row #
            #W_bin = '{0:04b}'.format(int(W[i,7-j].item()+0.001))
            if (W[i,7-j,kij].item()<0):
                W_temp=W[i,7-j,kij].item()+(2**bit_precision)
            else:
                W_temp=W[i,7-j,kij].item()
            W_bin = '{0:04b}'.format(int(W_temp+0.001))
            for k in range(bit_precision):
                file.write(str(W_bin[k]))        
#             file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close() #close file

In [22]:
weight_int.size()
W = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))
W.size()
W[0,:,0]

tensor([-1.,  2., -0.,  4.,  1.,  0.,  1., -4.], device='cuda:0',
       grad_fn=<SelectBackward>)

In [23]:
W[0,:,0]

tensor([-1.,  2., -0.,  4.,  1.,  0.,  1., -4.], device='cuda:0',
       grad_fn=<SelectBackward>)

In [24]:

bit_precision = 4
file = open('weight_dec.txt', 'w') #write to file
file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
file.write('#................#\n')


W_temp=0
for kij in range(9):
    for i in range(W.size(0)):  # Column #
        for j in range(W.size(1)): # row #
            #W_bin = '{0:04b}'.format(int(W[i,7-j].item()+0.001))
            if (W[i,7-j,kij].item()<0):
                W_temp=W[i,7-j,kij].item()+16
            else:
                W_temp=W[i,7-j,kij].item()
            W_dec = int(W_temp+0.001)
            if (W_dec>=8) :  
                W_dec = (W_dec - 16) 
            file.write(str(W_dec)) 
            print(i,7-j,kij, W_temp, W_dec, W[i,7-j,kij].item())
            print("\n")
            file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close() #close file

0 7 0 12.0 -4 -4.0


0 6 0 1.0 1 1.0


0 5 0 0.0 0 0.0


0 4 0 1.0 1 1.0


0 3 0 4.0 4 4.0


0 2 0 -0.0 0 -0.0


0 1 0 2.0 2 2.0


0 0 0 15.0 -1 -1.0


1 7 0 15.0 -1 -1.0


1 6 0 4.0 4 4.0


1 5 0 4.0 4 4.0


1 4 0 14.0 -2 -2.0


1 3 0 12.999999761581421 -3 -3.000000238418579


1 2 0 14.0 -2 -2.0


1 1 0 11.0 -5 -5.0


1 0 0 2.0 2 2.0


2 7 0 15.0 -1 -1.0


2 6 0 15.0 -1 -1.0


2 5 0 12.999999761581421 -3 -3.000000238418579


2 4 0 12.999999761581421 -3 -3.000000238418579


2 3 0 14.0 -2 -2.0


2 2 0 15.0 -1 -1.0


2 1 0 -0.0 0 -0.0


2 0 0 15.0 -1 -1.0


3 7 0 2.0 2 2.0


3 6 0 9.0 -7 -7.0


3 5 0 7.0 7 7.0


3 4 0 0.0 0 0.0


3 3 0 11.0 -5 -5.0


3 2 0 6.000000476837158 6 6.000000476837158


3 1 0 -0.0 0 -0.0


3 0 0 14.0 -2 -2.0


4 7 0 7.0 7 7.0


4 6 0 2.0 2 2.0


4 5 0 15.0 -1 -1.0


4 4 0 14.0 -2 -2.0


4 3 0 15.0 -1 -1.0


4 2 0 14.0 -2 -2.0


4 1 0 1.0 1 1.0


4 0 0 12.999999761581421 -3 -3.000000238418579


5 7 0 12.0 -4 -4.0


5 6 0 3.000000238418579 3 3.000000238418579


5 

In [25]:
output_int.size()
O = output_int[0]
O = torch.reshape(O, (O.size(0), -1))
O.size()


torch.Size([8, 16])

In [26]:
### Complete this cell ###



bit_precision = 16
file = open('psum.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(O.size(1)):  # time step
    for j in range(O.size(0)): # Column #
        if (O[7-j,i].item()<0):
            O_temp=O[7-j,i].item()+(2**bit_precision)
        else:
            O_temp=O[7-j,i].item()
        O_bin = '{0:016b}'.format(int(O_temp+0.001))
#         O_bin = str(int(O_temp+0.001))
        #psum_tile_bin = '{0:016b}'.format(int(psum_tile[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(O_bin[k])
#         file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file

In [27]:
O[:,0]

tensor([ 3.4000e+01,  8.0000e+00,  6.0000e+00, -4.0000e+00,  1.7000e+01,
         8.0000e+00, -9.5367e-07, -1.2000e+01], device='cuda:0',
       grad_fn=<SelectBackward>)

In [28]:
### Complete this cell ###



bit_precision = 16
file = open('psum_dec.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(O.size(1)):  # time step
    for j in range(O.size(0)): # Column #
        if (O[7-j,i].item()<0):
            O_temp=O[7-j,i].item()+(2**bit_precision)
        else:
            O_temp=O[7-j,i].item()
        O_dec = int(O_temp+0.001)
        if(O_dec>=2**(bit_precision-1)):
            O_dec=O_dec-(2**bit_precision)
        file.write(str(O_dec))
        file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file